In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
import librosa
import numpy as np    
import os
import signal
import subprocess
import json
import uuid
from IPython.display import HTML

Extractor (Input Validation, Chunking, Mapping From file_csv and annotations_csv to chunks)

In [3]:
from acoustic_knowledge_discovery import extractors

/Users/siyakamboj/Downloads/E4E/acoustic_knowledge_discovery/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
extractor = extractors.Extractor(
    "sample_inputs/file.csv", #file metadata csv
    "sample_inputs/anno.csv", #annoations csv
    "sample_inputs/inputDtory",#input directory with a files subdirectory for audio files & a XC-templates subdirectory for XC templates
    chunk_size = 5) #chunk size in seconds
chunkDS = extractor.forward()


Casting the dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

In [5]:
for row in chunkDS.chunk_ds["train"]:
    print(row)
chunkDS.chunk_ds

{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'SR': 48000.0, 'chunk_start': 0, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'SR': 48000.0, 'chunk_start': 5, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_5', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'SR': 48000.0, 'chunk_start': 10, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_10', 'Annotation': [], 'Confidence': []}
{'file_name': 'XC758442 - Common Blackbird - Turdus merula.wa

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence'],
        num_rows: 4
    })
})

Chunk DS Augmentations

In [6]:
def add_raw_audio_batched(batch):
    import librosa
    raws = []
    for fname, start in zip(batch["file_name"], batch["chunk_start"]):
        y, _ = librosa.load(
            extractor.base_dir / "files" / fname,
            offset=float(start),
            duration=float(extractor.chunk_size),
            sr=None
        )
        raws.append(y.astype(np.float32))
    # return list-of-arrays 
    return {"raw_audio": raws}


In [7]:
# Add raw audio column in dataset
chunkDS.chunk_ds = chunkDS.chunk_ds.map(add_raw_audio_batched, 
                                        batched=True, 
                                        num_proc=1) #increasing this number a small amount can make it faster
                                        # ONLY INCREASE ON MAC OR LINUX

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [8]:
chunkDS.chunk_ds #make sure that new column raw_audio is added

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'raw_audio'],
        num_rows: 4
    })
})

File-level Operations

In [9]:
from acoustic_knowledge_discovery.features.feature_preprocessing_pipeline import FeatureSequential
from acoustic_knowledge_discovery import features

THRESHOLD = 0.6

pipeline = FeatureSequential(
    features.TemplateMatching(
        extractor.base_dir / "files",
        extractor.base_dir / "XC-templates",
        THRESHOLD,
        chunk_size=extractor.chunk_size
    ),
    # features.anotherFunctionThatInheretsFromFeaturePreprocessor
)

chunkDS = pipeline.forward(chunkDS)  

1 total templates
Filtered down to 1 unique species templates
  XC758442 - Common Blackbird - Turdus merula: XC758442 - Common Blackbird - Turdus merula.wav


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Progress: 1/1 templates completed (100.0%)
Total matches found above the threshold of 0.6: 1


In [10]:
#verify properly loaded after feature extraction
for row in chunkDS.chunk_ds["train"]:
    print(row["Annotation"], row["Confidence"])

['owl'] [0.8999999761581421]
['owl'] [0.8999999761581421]
[] []
['XC758442 - Common Blackbird - Turdus merula'] [0.6061228513717651]


Chunk-level Operations

In [11]:
#Insert EGCI
egci = features.EGCI()
chunkDS= egci(chunkDS, num_proc=1)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [12]:
#verify EGCI column properly added
for row in chunkDS.chunk_ds["train"]:
    print(row["chunk_id"], row["EGCI"])
chunkDS.chunk_ds

XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0 [0.6825488344140658, 0.46425829958654957]
XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_5 [0.6520874513750399, 0.3366618587726256]
XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_10 [0.6374639178316074, 0.4597440843254349]
XC758442 - Common Blackbird - Turdus merula.wav_0 [0.7267189092284931, 0.3654319916606137]


DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'raw_audio', 'EGCI'],
        num_rows: 4
    })
})

Binning!

In [13]:
from acoustic_knowledge_discovery.postprocessing import MakeBins2dFloat
makebins = MakeBins2dFloat(chunkDS)
egci_bin={
    "low": ( (0.0, 34), (0.0,0.34) ), 
    "medium": ((0.34,0.67),(0.34,0.67)), 
    "high": ((0.67,1),(0.67,1))
}

chunkDS = makebins("EGCI", egci_bin)

Casting the dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

x is [0.68254883 0.65208745 0.63746392 0.72671891], x_lo is 0.0, x_hi is 34.0 
y is [0.4642583  0.33666186 0.45974408 0.36543199], y_lo is 0.0, y_hi is 0.34 
x is [0.68254883 0.65208745 0.63746392 0.72671891], x_lo is 0.34, x_hi is 0.67 
y is [0.4642583  0.33666186 0.45974408 0.36543199], y_lo is 0.34, y_hi is 0.67 
x is [0.68254883 0.65208745 0.63746392 0.72671891], x_lo is 0.67, x_hi is 1.0 
y is [0.4642583  0.33666186 0.45974408 0.36543199], y_lo is 0.67, y_hi is 1.0 


In [15]:
for row in chunkDS.chunk_ds["train"]:
    #label_str = chunkDS.chunk_ds["train"].features["EGCI"].int2str(row["EGCI"])
    print(row["chunk_id"], row["EGCI"])
chunkDS.chunk_ds

XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0 Other
XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_5 low
XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_10 medium
XC758442 - Common Blackbird - Turdus merula.wav_0 Other


DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'raw_audio', 'EGCI'],
        num_rows: 4
    })
})

In [16]:
chunkDS.chunk_ds["train"][0]

{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3',
 ' time_of_day': 'Morning',
 'season': 'Rainy',
 'species': ['Bird', 'Bat'],
 'SR': 48000.0,
 'chunk_start': 0,
 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0',
 'Annotation': ['owl'],
 'Confidence': [0.8999999761581421],
 'raw_audio': [-0.0024800426326692104,
  -0.0022246944718062878,
  -0.0019423383055254817,
  -1.5729674487374723e-05,
  -0.002592401346191764,
  -0.002900682156905532,
  0.0016191620379686356,
  -0.0013099081115797162,
  -0.0020179329439997673,
  0.0019927711691707373,
  0.002441997639834881,
  -0.0018278160132467747,
  -0.003584541380405426,
  -0.0008824444375932217,
  -0.006195208989083767,
  -0.005566014908254147,
  -0.00033756549237295985,
  -0.005473957397043705,
  -0.003265663515776396,
  -0.001258694683201611,
  -0.004577127285301685,
  -0.0028006895445287228,
  -0.0022822553291916847,
  -0.0004641521954908967,
  0.00019567309936974198,
  0.0012614486040547

In [17]:
# # Tests that MakeBins1dFloat does work, but it does not make sense to use it on this dataset
# from acoustic_knowledge_discovery.postprocessing import MakeBins1dFloat
# makebins1d = MakeBins1dFloat(chunkDS)
# OneDBin={
#     "low": (0, 5) , 
#     "medium": (5, 10), 
#     "high": (10, 15)
# }

# chunkDS = makebins1d("chunk_start", OneDBin)

# for row in chunkDS.chunk_ds["train"]:
#     print(row["chunk_id"], row["chunk_start"])
# chunkDS.chunk_ds

# AutoEncoder and Clustering

In [17]:
from acoustic_knowledge_discovery.features import AutoEncoderProcessor

autoEncoder = AutoEncoderProcessor(model_path = "sample_models/dim_10_muha.pt", num_dims=10)
chunkDS = autoEncoder(chunkDS)
chunkDS.chunk_ds

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'raw_audio', 'EGCI', 'sample_models/dim_10_muha.pt_embeddings'],
        num_rows: 4
    })
})

In [18]:
from acoustic_knowledge_discovery.postprocessing import Cluster_API

#HEY HEY HEY: You should note that more num_clusters means more possible nodes
# num_clusters should be some number that is less than your dataset
clustering_pipe = Cluster_API(chunkDS, num_clusters=2)
chunkDS = clustering_pipe("sample_models/dim_10_muha.pt_embeddings")
chunkDS.chunk_ds

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'raw_audio', 'EGCI', 'sample_models/dim_10_muha.pt_embeddings', 'sample_models/dim_10_muha.pt_embeddings_clusters'],
        num_rows: 4
    })
})

# Convert System to a graph representation

In [ ]:
def normalize(v):
    if isinstance(v, list):
        return v
    if isinstance(v, np.ndarray):   # convert numpy array to list
        return v.tolist()
    return [v]  # if it is scalar, then make it list

In [ ]:
test = chunkDS.chunk_ds.select_columns(["file_name", " time_of_day", "season", "species", "chunk_id", "Annotation", "EGCI", "sample_models/dim_10_muha.pt_embeddings_clusters"])
nodes = test["train"].to_pandas().melt()
nodes = nodes.assign(value=nodes["value"].apply(normalize)).explode("value", ignore_index=True) #necessary so list values are separated
nodes["id"] = nodes["variable"] + "_" + nodes["value"].apply(str)
nodes["group"] = nodes["variable"].apply(lambda x: list(nodes["variable"].unique()).index(x))
nodes[["id", "group"]].head()

,id,group
0,file_name_XC233970 - Black-chinned Sparrow - S...,0
1,file_name_XC233970 - Black-chinned Sparrow - S...,0
2,file_name_XC233970 - Black-chinned Sparrow - S...,0
3,file_name_XC758442 - Common Blackbird - Turdus...,0
4,time_of_day_Morning,1


In [21]:
edge_list = test["train"].to_pandas().melt(id_vars="file_name")
edge_list = edge_list.assign(value=edge_list["value"].apply(normalize)).explode("value", ignore_index=True)
edge_list = edge_list.dropna(subset=["value"])# drop Nan
edge_list["target"] = edge_list["variable"] + "_" + edge_list["value"].apply(str)
edge_list["source"] = "file_name" + "_" + edge_list["file_name"]
edge_list["value"] = 1
edge_list[["source", "target", "value"]].head()

,source,target,value
0,file_name_XC233970 - Black-chinned Sparrow - S...,time_of_day_Morning,1
1,file_name_XC233970 - Black-chinned Sparrow - S...,time_of_day_Morning,1
2,file_name_XC233970 - Black-chinned Sparrow - S...,time_of_day_Morning,1
3,file_name_XC758442 - Common Blackbird - Turdus...,time_of_day_Afternoon,1
4,file_name_XC233970 - Black-chinned Sparrow - S...,season_Rainy,1


In [22]:
import json

with open("acoustic_knowledge_discovery/d3-visualization/graph_representation.json", mode="w") as f:
    json.dump({
        "nodes": list(nodes[["id", "group"]].drop_duplicates().T.to_dict().values()),  
        "links":  list(edge_list[["source", "target", "value"]].T.to_dict().values()),
    }, f
)

# Visualization

In [23]:
!npm install

⠙⠹⠸⠼
up to date, audited 13 packages in 592ms
⠼
found 0 vulnerabilities
⠼

In [23]:
command = ["npx", "http-server", "-p", "8080"] 
target_directory = "acoustic_knowledge_discovery/d3-visualization/"

process = subprocess.Popen(
    command,
    cwd=target_directory,
    #shell=True,
    preexec_fn=os.setsid  
)

# Persist the group leader PID so you can kill it later even after kernel restarts & kill the entire tree
with open("/tmp/http_server_pid.json", "w") as f:
    json.dump({"pgid": process.pid}, f)

print(f"Server started. PGID={process.pid}")

Server started. PGID=27133


Starting up http-server, serving ./

http-server version: 14.1.1

http-server settings: 
CORS: disabled
Cache: 3600 seconds
Connection Timeout: 120 seconds
Directory Listings: visible
AutoIndex: visible
Serve GZIP Files: false
Serve Brotli Files: false
Default File Extension: none

Available on:
  http://127.0.0.1:8080
  http://192.168.1.243:8080
Hit CTRL-C to stop the server



In [24]:
#NOTE : You can also open up http://127.0.0.1:8080 in your browser to see it displayed on a larger screen
HTML(f"""
<iframe style="background-color: white;" 
        src="http://127.0.0.1:8080/?nocache={uuid.uuid4()}" 
        width="1200" height="1000"></iframe>
""")

[2025-09-11T23:11:05.708Z]  "GET /?nocache=35f72821-f90c-40f7-991e-7033b144697d" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.104.0 Chrome/138.0.7204.235 Electron/37.3.1 Safari/537.36"
[2025-09-11T23:11:05.743Z]  "GET /inspector.css" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.104.0 Chrome/138.0.7204.235 Electron/37.3.1 Safari/537.36"
[2025-09-11T23:11:05.745Z]  "GET /runtime.js" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.104.0 Chrome/138.0.7204.235 Electron/37.3.1 Safari/537.36"
[2025-09-11T23:11:05.746Z]  "GET /index.js" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.104.0 Chrome/138.0.7204.235 Electron/37.3.1 Safari/537.36"
[2025-09-11T23:11:05.764Z]  "GET /graph_representation.json" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.104.0 Ch

In [25]:
# Kill the entire process group (gracefully) to stop d3vis port
with open("/tmp/http_server_pid.json") as f:
    pgid = json.load(f)["pgid"]


os.killpg(pgid, signal.SIGTERM)

http-server stopped.


In [ ]:
#process.kill()